# Topic Modelling for Consumer Discritionary M&A Deal Ratonale's

In [84]:
import re
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from stop_words import safe_get_stop_words
from stop_words import get_stop_words
from textblob import TextBlob

In [85]:
consumer_dis = pd.read_excel("D:\\ISB\\Capstone Project\\Data\\1-CD_Deals\\1-Consumer Discretionary - New.xls",sheetname='MM_DealUnformattedReport')#,skiprows=7)
for i in range(0,len(consumer_dis)):
    try:
        consumer_dis['Deal Description'][i] = consumer_dis['Deal Description'][i].encode('utf8')
    except:
        print i

C:\Users\Rajiv2806\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
stop_words1 = set(stopwords.words('english'))
stop_words1 = " ".join(i.encode('utf-8') for i in stop_words1).split()
stop_words2 = ['million','approximately','july','march','august','february','october','company','usbased','press','release'
              ,'llc','usd','inc']
stop_words3 = list(get_stop_words('en'))
stop_words3 = " ".join(i.encode('utf-8') for i in stop_words3).split()
stop_words4 = list(safe_get_stop_words('unsupported language'))
stop_words5 = str(consumer_dis[['Target Company','Bidder Company']]) #,'Sellers','Target Ticker','Exchange:Ticker','Deal Responses','Target Company Industry']])
stop_words5 = list(pd.unique(stop_words5.split()))
stop_words = list(pd.unique(stop_words1+stop_words2+stop_words3+stop_words4+stop_words5))

stemming1 = WordNetLemmatizer()    #stemming2 = PorterStemmer()

In [78]:
unigram = []
bigram = []

for i in range(0,len(consumer_dis)):
    to_lowercase = consumer_dis['Deal Description'][i].lower()
    puncuation_removed = re.sub(r'[^\w\s]','',to_lowercase)
    newline_removed = re.sub('\n',' ',puncuation_removed)
    stopwords_removed = " ".join(j for j in newline_removed.split() if j not in stop_words)
    stemming_words1 = " ".join(stemming1.lemmatize(j) for j in stopwords_removed.split())
    numbers_removed = " ".join(re.sub("\d+","",i) for i in stemming_words1.split())
    letters_removed = " ".join(j for j in numbers_removed.split() if len(j) > 2)
    trimmed = letters_removed.strip()
    unigram.append(trimmed)
    
    a = ""
    for j in range(0,(len(trimmed.split())-1)):
        empty = trimmed.split()[j]+"_"+trimmed.split()[j+1]
        a = a+" "+empty
    a = a.strip()
    bigram.append(a)
    
data = [str(bigram[i])+" "+str(unigram[i]) for i in range(len(bigram))]

n_features = 1000

# tf or count matrix
# for topic modelling LDA can only use raw term counts because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=8, max_features=n_features, stop_words='english')
tf = tf_vectorizer.fit_transform(data)
tf_feature_names = tf_vectorizer.get_feature_names()

# tf-idf matrix
# for topic modelling NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=8, max_features=n_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

n_topics = 8 #input("No of topics:")

nmf = NMF(n_components=n_topics,random_state=1,alpha=.1,l1_ratio=.5,init='nndsvd')
nmf = nmf.fit(tfidf)

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
lda = lda.fit(tf)

# print "The Shape of NMF Matrix:",nmf.components_.shape
# print "The Shape of LDA Matrix:",lda.components_.shape

def display_topics(model, feature_names, no_top_words):
    if str(model)[0:3] == "NMF":
        print "NMF MODEL"
    elif str(model)[0:3] == "Lat":
        print "LDA MODEL"
        
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx+1)
        print " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
    print "="*100
    
no_top_words = 15

print "Topics For Consumer Discretionary"
print "="*100

display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tfidf_feature_names, no_top_words)

Topics For Consumer Discretionary
NMF MODEL
Topic 1:
product brand acquisition marketing service transaction engaged acquired undisclosed undisclosed_consideration business manufacturer market continue portfolio
Topic 2:
dealership automotive automotive_dealership car auto private_individual individual car_dealership individual_interest dealership_undisclosed dealership_acquired nissan chevrolet private group
Topic 3:
share sec filed_sec filed form_filed form headquartered exhibit transaction expected price closing agreement cash condition
Topic 4:
medium television station communication publishing newspaper digital broadcasting group content radio new_medium entertainment gray medium_group
Topic 5:
cocacola bottling refreshment cocacola_bottling beverage soft drink bottled asset distribution cocacola_refreshment bottling_consolidated refreshment_usa consolidated distribution_asset
Topic 6:
capital venture partner firm equity private_equity private equity_firm capital_partner venture_c

In [90]:
topic_output_lda = np.column_stack((data,lda.transform(tf)))
topic_output_nmf = np.column_stack((data,nmf.transform(tfidf)))
df1 = pd.DataFrame(topic_output_lda,columns=['Deal Rationale','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8'])
df2 = pd.DataFrame(topic_output_nmf,columns=['Deal Rationale','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8'])
df1["LDA Topic"] = df1[df1.columns[1:9]].idxmax(axis=1)
df2["NMF Topic"] = df2[df2.columns[1:9]].idxmax(axis=1)

consumer_dis["NMF TopicID"] = df2[df2.columns[1:9]].idxmax(axis=1)
consumer_dis["LDA TopicID"] = df1[df1.columns[1:9]].idxmax(axis=1)

consumer_dis.to_csv("D:/ISB/Capstone Project/Data/1-CD_Deals/Consumer Discretionary - Topics.csv",index=False,encoding='utf-8')

## Summary:

Topic modelling on Consumer Discritionary acquisitions is done using the LDA and NMF methods.

Results for the the Deal rationale, for 8 topics of both the models are:

**Topics from Deal Description are:**

**NMF MODEL:** *Except Topic 1 others are interpreted*

Topic 1: product brand acquisition marketing service transaction engaged acquired undisclosed undisclosed_consideration business manufacturer market continue portfolio 

Topic 2: dealership automotive automotive_dealership car auto private_individual individual car_dealership individual_interest dealership_undisclosed dealership_acquired nissan chevrolet private group --- Automobile Industry

Topic 3: share sec filed_sec filed form_filed form headquartered exhibit transaction expected price closing agreement cash condition  ---deals aggrement related

Topic 4: medium television station communication publishing newspaper digital broadcasting group content radio new_medium entertainment gray medium_group  ---Media groups

Topic 5: cocacola bottling refreshment cocacola_bottling beverage soft drink bottled asset distribution cocacola_refreshment bottling_consolidated refreshment_usa consolidated distribution_asset  ---beverages Industry

Topic 6: capital venture partner firm equity private_equity private equity_firm capital_partner venture_capital capital_firm firm_undisclosed partner_private undisclosed investor   ---related to firm financials

Topic 7: food product conagra frozen snack meat ingredient food_holding food_corporation food_product producer pet vegetable sauce organic   ---food indistry

Topic 8: store convenience convenience_store retail fuel operator supermarket chain gas retailer grocery market seven operates station  ---retail indus


**LDA MODEL**  ---Looks tough to interpret the topics

Topic 1: product food acquisition acquired transaction undisclosed undisclosed_consideration engaged source business link brand source_link company employee

Topic 2: cocacola beverage bottling distribution asset coffee cocacola_bottling consolidated water drink marketing refreshment soft tea bottling_consolidated

Topic 3: transaction acquisition expected food filed store form sec filed_sec form_filed acquire brand listed update cash

Topic 4: dealership automotive acquired undisclosed undisclosed_consideration group automotive_dealership car auto private acquisition individual private_individual individual_interest car_dealership

Topic 5: share transaction form filed sec filed_sec form_filed headquartered price exhibit agreement new closing offer expected

Topic 6: medium acquisition service digital marketing acquired transaction group new source engaged source_link link business publishing

Topic 7: capital partner equity firm private private_equity equity_firm acquired venture undisclosed acquisition undisclosed_consideration transaction portfolio group

Topic 8: broadcasting station television nexstar medium television_station nexstar_broadcasting broadcasting_group general group television_broadcasting communication fcc group_form local